# #010 - Monte Carlo Simulations for a Portfolio

## libs

In [1]:
#import Libraries
import pandas as pd
from pandas_datareader import data as pdr
import numpy as np
import random

## Functions

In [2]:
def rand_weights(n):
    ''' Produces n random weights that sum to 1 '''
    k = np.random.rand(n)
    return k / sum(k)

In [3]:
def run_portfolio(df, weights, initial_investment, risk_free):
    ''' Performs an asset allocation and calc Portfolio Metrics'''
    # The function returns: 
    # (0) Portfolio Data Frame 
    # (1) Expected portfolio return 
    # (2) Expected volatility 
    # (3) Sharpe ratio 
    # (4) Return on investment 
    # (5) Final portfolio value in dollars
    
    # Perform asset allocation using the random weights (sent as arguments to the function)
    portfolio_df =  df.copy()
    # Scale stock prices using the "price_scaling"
    scaled_df = df.copy()
    for i in df.columns[0:]:
        scaled_df[i] = df[i]/df[i][0]
    #enumerate method links Stocks tickers in columns along with a counter position weight (i), like an index
    for i, stock in enumerate(scaled_df):
        portfolio_df[stock] = weights[i] * scaled_df[stock]  * initial_investment
    
    # Sum up all values and place the result in a new column titled "portfolio value [$]" 
    portfolio_df['Total Value [$]'] = portfolio_df.sum(axis = 1, numeric_only = True)
    # Calculate the portfolio percentage daily return and replace NaNs with zeros
    portfolio_df['Daily Return [%]'] = portfolio_df['Total Value [$]'].pct_change(1) * 100 
    portfolio_df.replace(np.nan, 0, inplace = True)
    # Calculate the return on the investment = last final value of the portfolio compared to its initial value
    return_on_investment = ((portfolio_df['Total Value [$]'][-1:] - portfolio_df['Total Value [$]'][0]) 
                            / portfolio_df['Total Value [$]'][0]) * 100
    
    # Daily change of every stock in the portfolio 
    portfolio_daily_return_df = portfolio_df.drop(columns = ['Total Value [$]', 'Daily Return [%]'])
    portfolio_daily_return_df = portfolio_daily_return_df.pct_change(1)
    portfolio_daily_return_df.replace(np.nan, 0, inplace = True)  
  
    # Portfolio Expected Return formula
    expected_portfolio_return = np.sum(weights * portfolio_daily_return_df.mean() ) * 252
  
    # Portfolio standard deviation, volatility (risk)
    covariance = portfolio_daily_return_df.cov() * 252 
    expected_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance, weights)))
    
    # Calculate Sharpe ratio
    sharpe_ratio = (expected_portfolio_return - risk_free)/expected_volatility 
    
    return expected_portfolio_return, expected_volatility, sharpe_ratio, portfolio_df['Total Value [$]'][-1:].values[0], return_on_investment.values[0]

## 10.1 Read data and store on DataFrame

In [4]:
file_name =          input('Input the CSV file name: ')
initial_investment = int(input('Input the initial investment: '))
risk_free =          float(input('Input the risk free annual rate: ')) # https://ycharts.com/ind icators/10_year_treasury_rate 
sim_runs =           int(input('Input how many runs to perform on Monte Carlo Simulation: '))

# file_name =          'MAG7'
# initial_investment = 1000000
# risk_free =          0.039
# sim_runs =           1000

Input the CSV file name: MAG7
Input the initial investment: 1000000
Input the risk free annual rate: 0.039
Input how many runs to perform on Monte Carlo Simulation: 1000


In [5]:
#read CSV file
df = pd.read_csv(file_name)
#The code imports a DataFrame with num index [1,2,3...], this line replace the colum Date to Index
df.set_index(['Date'], inplace = True)

## 10.2 Perform Monte Carlo Simulations

In [6]:
# Set the number of simulation runs
n = n_assets = len(df.columns)

# Array to store all weights
weights_runs = np.zeros((sim_runs, n))
# Array to store all Sharpe ratios
sharpe_ratio_runs = np.zeros(sim_runs)
# Array to store all expected returns
expected_portfolio_returns_runs = np.zeros(sim_runs)
# Array to store all volatility values
volatility_runs = np.zeros(sim_runs)
# Array to store all returns on investment
return_on_investment_runs = np.zeros(sim_runs)
# Array to store all final portfolio values
final_value_runs = np.zeros(sim_runs)

In [7]:
for i in range(sim_runs):
    # Generate random weights 
    weights = rand_weights(n)
    # Store the weights
    weights_runs[i,:] = weights
    
    # Call 'run_portfolio' function and store Sharpe ratio, return and volatility 
    expected_portfolio_returns_runs[i], volatility_runs[i], sharpe_ratio_runs[i], final_value_runs[i], return_on_investment_runs[i] = run_portfolio(df, weights, initial_investment, risk_free)
    print("Simulation Run = {}".format(i))   
    print("Weights = {}, Final Value = ${:.2f}, Sharpe Ratio = {:.2f}".format(weights_runs[i].round(3), final_value_runs[i], sharpe_ratio_runs[i]))   
    print('\n')

Simulation Run = 0
Weights = [0.171 0.284 0.062 0.024 0.126 0.239 0.094], Final Value = $6098029.90, Sharpe Ratio = 1.06


Simulation Run = 1
Weights = [0.099 0.104 0.223 0.069 0.158 0.297 0.05 ], Final Value = $6382496.09, Sharpe Ratio = 1.07


Simulation Run = 2
Weights = [0.016 0.244 0.143 0.205 0.072 0.277 0.042], Final Value = $5686777.77, Sharpe Ratio = 0.95


Simulation Run = 3
Weights = [0.241 0.152 0.142 0.099 0.024 0.277 0.066], Final Value = $6417649.60, Sharpe Ratio = 1.09


Simulation Run = 4
Weights = [0.002 0.099 0.087 0.235 0.226 0.231 0.12 ], Final Value = $6002388.94, Sharpe Ratio = 1.04


Simulation Run = 5
Weights = [0.192 0.124 0.158 0.156 0.121 0.061 0.189], Final Value = $5020815.10, Sharpe Ratio = 1.03


Simulation Run = 6
Weights = [0.083 0.257 0.192 0.17  0.153 0.006 0.14 ], Final Value = $3777149.38, Sharpe Ratio = 0.87


Simulation Run = 7
Weights = [0.16  0.146 0.065 0.19  0.112 0.082 0.245], Final Value = $5537627.00, Sharpe Ratio = 1.05


Simulation Run =

Simulation Run = 67
Weights = [0.224 0.045 0.246 0.049 0.032 0.245 0.159], Final Value = $6804210.93, Sharpe Ratio = 1.15


Simulation Run = 68
Weights = [0.193 0.134 0.225 0.24  0.059 0.075 0.076], Final Value = $4264659.52, Sharpe Ratio = 0.92


Simulation Run = 69
Weights = [0.11  0.108 0.192 0.054 0.268 0.059 0.209], Final Value = $5161118.96, Sharpe Ratio = 1.05


Simulation Run = 70
Weights = [0.056 0.185 0.144 0.094 0.216 0.08  0.225], Final Value = $5240935.38, Sharpe Ratio = 1.02


Simulation Run = 71
Weights = [0.204 0.099 0.19  0.167 0.238 0.022 0.081], Final Value = $4016001.19, Sharpe Ratio = 0.95


Simulation Run = 72
Weights = [0.028 0.145 0.161 0.18  0.242 0.078 0.165], Final Value = $4768753.71, Sharpe Ratio = 0.98


Simulation Run = 73
Weights = [0.165 0.059 0.06  0.204 0.251 0.107 0.153], Final Value = $5375455.45, Sharpe Ratio = 1.06


Simulation Run = 74
Weights = [0.328 0.053 0.066 0.219 0.021 0.185 0.128], Final Value = $6138280.67, Sharpe Ratio = 1.10


Simulati

Simulation Run = 137
Weights = [0.045 0.256 0.171 0.097 0.219 0.07  0.142], Final Value = $4470771.37, Sharpe Ratio = 0.94


Simulation Run = 138
Weights = [0.239 0.107 0.167 0.227 0.002 0.183 0.074], Final Value = $5455372.10, Sharpe Ratio = 1.02


Simulation Run = 139
Weights = [0.155 0.086 0.175 0.206 0.187 0.094 0.097], Final Value = $4719897.69, Sharpe Ratio = 0.99


Simulation Run = 140
Weights = [0.027 0.249 0.034 0.307 0.282 0.094 0.008], Final Value = $3746235.09, Sharpe Ratio = 0.80


Simulation Run = 141
Weights = [0.132 0.135 0.097 0.213 0.075 0.156 0.193], Final Value = $5839577.08, Sharpe Ratio = 1.05


Simulation Run = 142
Weights = [0.29  0.285 0.216 0.024 0.115 0.045 0.025], Final Value = $3816883.67, Sharpe Ratio = 0.88


Simulation Run = 143
Weights = [0.1   0.006 0.184 0.264 0.261 0.154 0.031], Final Value = $4875843.68, Sharpe Ratio = 0.97


Simulation Run = 144
Weights = [0.227 0.082 0.079 0.256 0.135 0.026 0.195], Final Value = $4799619.84, Sharpe Ratio = 1.01




Simulation Run = 205
Weights = [0.03  0.231 0.184 0.166 0.204 0.056 0.13 ], Final Value = $4187271.36, Sharpe Ratio = 0.91


Simulation Run = 206
Weights = [0.127 0.199 0.153 0.2   0.014 0.196 0.11 ], Final Value = $5544507.04, Sharpe Ratio = 1.00


Simulation Run = 207
Weights = [0.166 0.166 0.041 0.311 0.022 0.11  0.184], Final Value = $5265891.95, Sharpe Ratio = 0.99


Simulation Run = 208
Weights = [0.146 0.066 0.228 0.302 0.038 0.068 0.152], Final Value = $4629710.03, Sharpe Ratio = 0.95


Simulation Run = 209
Weights = [0.12  0.296 0.03  0.013 0.031 0.343 0.165], Final Value = $7476024.43, Sharpe Ratio = 1.11


Simulation Run = 210
Weights = [0.28  0.061 0.036 0.262 0.065 0.291 0.005], Final Value = $6292512.19, Sharpe Ratio = 1.05


Simulation Run = 211
Weights = [0.06  0.074 0.114 0.302 0.187 0.045 0.219], Final Value = $4833897.00, Sharpe Ratio = 0.98


Simulation Run = 212
Weights = [0.061 0.147 0.172 0.148 0.056 0.067 0.348], Final Value = $5832320.12, Sharpe Ratio = 1.03




Simulation Run = 273
Weights = [0.208 0.201 0.224 0.057 0.2   0.014 0.097], Final Value = $3966275.72, Sharpe Ratio = 0.94


Simulation Run = 274
Weights = [0.033 0.015 0.087 0.219 0.373 0.217 0.056], Final Value = $5696812.16, Sharpe Ratio = 1.04


Simulation Run = 275
Weights = [0.078 0.048 0.205 0.304 0.175 0.085 0.106], Final Value = $4491881.81, Sharpe Ratio = 0.94


Simulation Run = 276
Weights = [0.226 0.229 0.176 0.161 0.021 0.022 0.165], Final Value = $4333123.63, Sharpe Ratio = 0.94


Simulation Run = 277
Weights = [0.09  0.099 0.189 0.221 0.187 0.06  0.154], Final Value = $4603024.08, Sharpe Ratio = 0.97


Simulation Run = 278
Weights = [0.22  0.067 0.043 0.188 0.02  0.24  0.223], Final Value = $7138511.18, Sharpe Ratio = 1.15


Simulation Run = 279
Weights = [0.102 0.22  0.042 0.194 0.119 0.083 0.239], Final Value = $5326001.20, Sharpe Ratio = 1.01


Simulation Run = 280
Weights = [0.037 0.229 0.029 0.227 0.166 0.071 0.241], Final Value = $5099887.38, Sharpe Ratio = 0.98




...

Simulation Run = 678
Weights = [0.176 0.016 0.086 0.117 0.427 0.05  0.127], Final Value = $4867635.28, Sharpe Ratio = 1.06


Simulation Run = 679
Weights = [0.175 0.158 0.14  0.163 0.115 0.116 0.133], Final Value = $5128834.90, Sharpe Ratio = 1.02


Simulation Run = 680
Weights = [0.001 0.179 0.122 0.234 0.172 0.139 0.152], Final Value = $5136209.70, Sharpe Ratio = 0.97


Simulation Run = 681
Weights = [0.194 0.155 0.075 0.184 0.051 0.211 0.13 ], Final Value = $6077176.61, Sharpe Ratio = 1.07


Simulation Run = 682
Weights = [0.23  0.254 0.066 0.187 0.064 0.041 0.158], Final Value = $4520047.47, Sharpe Ratio = 0.95


Simulation Run = 683
Weights = [0.007 0.186 0.2   0.229 0.222 0.078 0.078], Final Value = $4028167.84, Sharpe Ratio = 0.87


Simulation Run = 684
Weights = [0.24  0.005 0.055 0.041 0.214 0.297 0.147], Final Value = $7547773.36, Sharpe Ratio = 1.21


Simulation Run = 685
Weights = [0.239 0.248 0.159 0.072 0.016 0.154 0.113], Final Value = $5380639.83, Sharpe Ratio = 1.02




Simulation Run = 744
Weights = [0.209 0.203 0.154 0.222 0.073 0.123 0.016], Final Value = $4350314.73, Sharpe Ratio = 0.90


Simulation Run = 745
Weights = [0.053 0.229 0.131 0.042 0.217 0.116 0.212], Final Value = $5510279.61, Sharpe Ratio = 1.04


Simulation Run = 746
Weights = [0.162 0.23  0.223 0.066 0.134 0.026 0.158], Final Value = $4329167.96, Sharpe Ratio = 0.95


Simulation Run = 747
Weights = [0.273 0.055 0.137 0.099 0.278 0.085 0.074], Final Value = $4880336.44, Sharpe Ratio = 1.05


Simulation Run = 748
Weights = [0.155 0.069 0.06  0.236 0.142 0.176 0.162], Final Value = $5995602.28, Sharpe Ratio = 1.08


Simulation Run = 749
Weights = [0.005 0.164 0.187 0.17  0.124 0.19  0.161], Final Value = $5715304.48, Sharpe Ratio = 1.02


Simulation Run = 750
Weights = [0.017 0.227 0.314 0.19  0.13  0.012 0.11 ], Final Value = $3459926.51, Sharpe Ratio = 0.82


Simulation Run = 751
Weights = [0.204 0.15  0.16  0.141 0.021 0.194 0.13 ], Final Value = $5897119.98, Sharpe Ratio = 1.06




Simulation Run = 810
Weights = [0.164 0.051 0.118 0.05  0.162 0.183 0.273], Final Value = $6970438.30, Sharpe Ratio = 1.16


Simulation Run = 811
Weights = [0.364 0.129 0.215 0.064 0.084 0.082 0.062], Final Value = $4710800.74, Sharpe Ratio = 1.01


Simulation Run = 812
Weights = [0.211 0.068 0.28  0.066 0.111 0.167 0.097], Final Value = $5574546.08, Sharpe Ratio = 1.07


Simulation Run = 813
Weights = [0.218 0.173 0.075 0.218 0.129 0.    0.187], Final Value = $4387086.40, Sharpe Ratio = 0.96


Simulation Run = 814
Weights = [0.139 0.066 0.15  0.156 0.091 0.151 0.247], Final Value = $6274858.83, Sharpe Ratio = 1.10


Simulation Run = 815
Weights = [0.132 0.252 0.024 0.009 0.261 0.116 0.206], Final Value = $5695657.98, Sharpe Ratio = 1.06


Simulation Run = 816
Weights = [0.28  0.08  0.042 0.064 0.244 0.126 0.164], Final Value = $5932071.22, Sharpe Ratio = 1.13


Simulation Run = 817
Weights = [0.082 0.291 0.037 0.131 0.073 0.188 0.197], Final Value = $5996899.32, Sharpe Ratio = 1.03




Simulation Run = 876
Weights = [0.198 0.008 0.17  0.185 0.047 0.119 0.274], Final Value = $6247921.32, Sharpe Ratio = 1.11


Simulation Run = 877
Weights = [0.135 0.254 0.051 0.135 0.011 0.148 0.265], Final Value = $6145728.53, Sharpe Ratio = 1.05


Simulation Run = 878
Weights = [0.24  0.122 0.123 0.244 0.01  0.186 0.075], Final Value = $5493710.38, Sharpe Ratio = 1.01


Simulation Run = 879
Weights = [0.169 0.009 0.071 0.255 0.236 0.216 0.043], Final Value = $5755932.37, Sharpe Ratio = 1.05


Simulation Run = 880
Weights = [0.087 0.3   0.17  0.075 0.157 0.11  0.101], Final Value = $4599093.23, Sharpe Ratio = 0.94


Simulation Run = 881
Weights = [0.131 0.154 0.137 0.16  0.101 0.188 0.128], Final Value = $5737130.14, Sharpe Ratio = 1.05


Simulation Run = 882
Weights = [0.153 0.23  0.056 0.159 0.104 0.116 0.183], Final Value = $5370888.82, Sharpe Ratio = 1.02


Simulation Run = 883
Weights = [0.082 0.144 0.34  0.097 0.25  0.059 0.029], Final Value = $3745926.08, Sharpe Ratio = 0.87




Simulation Run = 942
Weights = [0.164 0.201 0.187 0.011 0.199 0.087 0.15 ], Final Value = $5021841.98, Sharpe Ratio = 1.03


Simulation Run = 943
Weights = [0.056 0.168 0.302 0.038 0.344 0.016 0.076], Final Value = $3679302.94, Sharpe Ratio = 0.89


Simulation Run = 944
Weights = [0.089 0.067 0.245 0.089 0.269 0.064 0.177], Final Value = $4961679.52, Sharpe Ratio = 1.04


Simulation Run = 945
Weights = [0.007 0.011 0.28  0.265 0.122 0.162 0.154], Final Value = $5462556.07, Sharpe Ratio = 1.01


Simulation Run = 946
Weights = [0.106 0.024 0.059 0.276 0.225 0.272 0.038], Final Value = $6128264.84, Sharpe Ratio = 1.04


Simulation Run = 947
Weights = [0.214 0.217 0.063 0.021 0.101 0.098 0.287], Final Value = $6103253.66, Sharpe Ratio = 1.08


Simulation Run = 948
Weights = [0.074 0.032 0.188 0.165 0.113 0.302 0.125], Final Value = $6894688.88, Sharpe Ratio = 1.12


Simulation Run = 949
Weights = [0.231 0.119 0.005 0.119 0.145 0.147 0.234], Final Value = $6376640.29, Sharpe Ratio = 1.13


